
## Mathemathics and programing for AI


### Tasks 1, 3 and 4

In [ ]:
from __future__ import print_function, division
from builtins import range
import numpy as np
from scipy.special import expit
import sys
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import loadmat
import os
import scipy.stats as stats


path="."

df_svhn_train = loadmat('train_32x32.mat')
df_svhn_test = loadmat('test_32x32.mat')
df_svhn_extra = loadmat('extra_32X32.mat')

print(df_svhn_test['X'].shape)
print(df_svhn_train['X'].shape)
print(df_svhn_extra['X'].shape)


X_train, y_train = df_svhn_train['X'], df_svhn_train['y']
X_test, y_test = df_svhn_test['X'], df_svhn_test['y']
X_extra, y_extra = df_svhn_extra['X'], df_svhn_extra['y']

X_train, y_train = X_train.transpose((3,0,1,2)), y_train[:,0]
X_test, y_test = X_test.transpose((3,0,1,2)), y_test[:,0]
X_extra, y_extra = X_extra.transpose((3,0,1,2)), y_extra[:,0]

print("train set", X_train.shape, y_train.shape)
print("test set",X_test.shape, y_test.shape)
print("extra set",X_extra.shape, y_extra.shape)

###############
#Original shapes train, test and extra datasets: (32,32,3,73257) and (32,32,3,26032)
###############

fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True,)
ax = ax.flatten()
for i in range(25):
    img = X_train[i].reshape(32,32,3)
    ax[i].imshow(img)
plt.show()

###############
#Concatenating test and extra dataset
###############
X_train = np.append(X_train, X_extra, axis = 0)
y_train = np.append(y_train, y_extra, axis = 0)

print(X_train.shape)
print(y_train.shape)
size_before = (X_train.nbytes, X_test.nbytes)
print("Size before converting to grayscale: ", size_before)

# =============================================================================
# Relabeling 10=0  and Normalizing data
# =============================================================================
print("Unique labels", np.unique(y_train) )
n_labels = 10
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

suffixes = ['B', 'KB', 'MB', 'GB']

def readablesize(nbytes):
    """printin size in reable format"""
    if nbytes == 0: return '0 B'
    i = 0
    while nbytes >= 1024:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

def rgbgray(rgbimages):
    """Convert images from rbg to grayscale
    """
    return np.expand_dims(np.dot(rgbimages, [0.2989, 0.5870, 0.1140]), axis=3)

#Transform the images to greyscale
X_train_g = rgbgray(X_train).astype(np.float32)
X_test_g = rgbgray(X_test).astype(np.float32)
# Size after transformation
size_after = (X_train_g.nbytes, X_test_g.nbytes)

print("Dimensions")
print("Training set",X_train_g.shape)
print("Test set", X_test_g.shape)
print('')

print("Dataset Size")
print("Training set", readablesize(size_before[0]), readablesize(size_after[0]))
print("Test set", readablesize(size_before[1]), readablesize(size_after[1]))

fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True,)
ax = ax.flatten()
for i in range(25):
    img = X_train_g[i].reshape(32,32)
    ax[i].imshow(img, cmap='Greys')

plt.show()

fact = 0.99/255
X_train = X_train_g.flatten().reshape(604388, 1024)*fact + 0.01
X_test = X_test_g.flatten().reshape(26032, 1024)*fact + 0.01
print(X_train.shape, X_test.shape)


### Saving files

Using numpy.savez_compressed to save images array in numpy format npz


In [ ]:
np.savez_compressed('svhn_scaled.npz', 
                    X_train=X_train,
                    y_train=y_train,
                    X_test=X_test,
                    y_test=y_test)

svhn = np.load('svhn_scaled.npz')
svhn.files

X_train, y_train, X_test, y_test = [svhn[f] for f in ['X_train', 'y_train', 
                                    'X_test', 'y_test']]

X_test.shape

In [ ]:
import numpy as np
import sys


class neuralnetwork(object):
    """ Feedforward neural network / Multi-layer perceptron classifier
    activation of outputlayer: act_out"""
    def __init__(self, n_hidden1=30,n_hidden2=30,l2=0., epochs=100, eta=0.001,
                 shuffle=True, minibatch_size=1, seed=None):

        self.random = np.random.RandomState(seed)
        self.n_hidden1 = n_hidden1
        self.n_hidden2 = n_hidden2 
        self.l2 = l2
        self.epochs = epochs
        self.eta = eta
        self.shuffle = shuffle
#         self.act_out = act_out
        self.minibatch_size = minibatch_size

    def _onehot(self, y, n_classes):
        """Encode labels into one-hot representation"""
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.
        return onehot.T
    
    def _sigmoid(self, z):
        """Compute logistic function (sigmoid)"""
        return 1. / (1. + np.exp(-np.clip(z, -250, 250)))
     
    def _relu(self, z):
        """Implements leakyrelu using a negative slope alpha"""
        alpha = 0.01
        return np.maximum(alpha*z, z)  
    
    def _relu_d(self, z):
        """Computes relu derivative with alpha"""
        alpha = 0.01
        relu_d = np.where(z <= 0, alpha, 1.0) 
        return relu_d
 
    def _softmax(X):
        """Compute softmax values for each sets of scores in x."""
        ex = np.exp(x - np.max(x))
        return ex / ex.sum()

    def _forward(self, X):
        """Compute forward propagation step"""
      
        z_h1 = np.dot(X, self.w_h1) + self.b_h1
        a_h1 = self._relu(z_h1)

        z_h2 = np.dot(a_h1, self.w_h2) + self.b_h2
        a_h2 = self._relu(z_h2)
        z_out = np.dot(a_h2, self.w_out) + self.b_out
        a_out = self._sigmoid(z_out)

        return z_h1, a_h1, z_h2, a_h2, z_out, a_out

    def _cost(self, y_enc, a_out):
        """Compute cost function with L2 regularizer """
        L2_term = (self.l2 *
                   (np.sum(self.w_h1 ** 2.) + np.sum(self.w_h2 ** 2.) + np.sum(self.w_out ** 2.)))

        term1 = -y_enc * (np.log(a_out))
        term2 = (1. - y_enc) * np.log(1. - a_out)
        cost = np.sum(term1 - term2) + L2_term
        return cost   

    def _predict(self, X):
        """Labels prediction"""
        z_h1, a_h1, z_h2, a_h2, z_out, a_out = self._forward(X)
        y_pred = np.argmax(a_out, axis=1)
        print("y_pred",y_pred)
        return y_pred
    
    def fit(self, X_train, y_train, X_valid, y_valid):
        """ Learn weights from training data """
        
        n_output = np.unique(y_train).shape[0]  # number of labels
        n_features = X_train.shape[1]

        ########################
        # Weight initialization
        ########################

        self.b_h1 = np.zeros(self.n_hidden1)
        self.w_h1 = self.random.normal(loc=0.0, scale=0.1,size=(n_features, self.n_hidden1))
        self.b_h2 = np.zeros(self.n_hidden2)
        self.w_h2 = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden1, self.n_hidden2))
        
        # weights for hidden -> output
        self.b_out = np.zeros(n_output)
        self.w_out = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden2, n_output))
        
        epoch_strlen = len(str(self.epochs))  # for progress formatting
        self.eval_ = {'cost': [], 'train_acc': [], 'valid_acc': []}

        y_train_enc = self._onehot(y_train, n_output)

        
        # iterate over training epochs
        for i in range(self.epochs):

            # iterate over minibatches
            samples = np.arange(X_train.shape[0])
            print("samples:",samples)

            if self.shuffle:
                self.random.shuffle(samples)

            for start_idx in range(0, samples.shape[0] - self.minibatch_size +
                                   1, self.minibatch_size):
                batch_idx = samples[start_idx:start_idx + self.minibatch_size]

              
                # forward propagation
             
                z_h1, a_h1, z_h2, a_h2, z_out, a_out = self._forward(X_train[batch_idx])

               
                # Backpropagation
               
                sigma_out = a_out - y_train_enc[batch_idx]
                relu_derivative2 = self._relu_d(a_h2)
                sigma_h2 = (np.dot(sigma_out, self.w_out.T) * relu_derivative2)
                relu_derivative1 = self._relu_d(a_h1)
                sigma_h1 = (np.dot(sigma_h2, self.w_h2.T) * relu_derivative1)
                
                grad_w_h2 = np.dot(a_h1.T, sigma_h2)
                grad_w_h1 = np.dot(X_train[batch_idx].T, sigma_h1)
                grad_b_h2 = np.sum(sigma_h2, axis=0)
                grad_b_h1 = np.sum(sigma_h1, axis=0)

                grad_w_out = np.dot(a_h2.T, sigma_out)
                grad_b_out = np.sum(sigma_out, axis=0)

                delta_w_h2 = (grad_w_h2 + self.l2*self.w_h2)
                delta_w_h1 = (grad_w_h1 + self.l2*self.w_h1)
                delta_w_out = (grad_w_out + self.l2*self.w_out)
                
                delta_b_h2 = grad_b_h2 # bias is not regularized
                delta_b_h1 = grad_b_h1
                delta_b_out = grad_b_out
                
                self.w_h2 -= self.eta * delta_w_h2
                self.w_h1 -= self.eta * delta_w_h1
                self.w_out -= self.eta * delta_w_out
                self.b_h2 -= self.eta * delta_b_h2
                self.b_h1 -= self.eta * delta_b_h1           
                self.b_out -= self.eta * delta_b_out

        
            # Evaluation
      
            z_h1, a_h1, z_h2, a_h2, z_out, a_out = self._forward(X_train)
            
            cost = self._cost(y_enc=y_train_enc,
                                      a_out=a_out)

            y_train_pred = self._predict(X_train)
            y_valid_pred = self._predict(X_valid)

            train_acc = ((np.sum(y_train == y_train_pred)).astype(np.float) /
                         X_train.shape[0])
            valid_acc = ((np.sum(y_valid == y_valid_pred)).astype(np.float) /
                         X_valid.shape[0])
            print("test acc", valid_acc)
            sys.stderr.write('\r%0*d/%d | Cost: %.2f '
                             '| Train/Valid Acc.: %.2f%%/%.2f%% ' %
                             (epoch_strlen, i+1, self.epochs, cost,
                              train_acc*100, valid_acc*100))
            sys.stderr.flush()

            self.eval_['cost'].append(cost)
            self.eval_['train_acc'].append(train_acc)
            self.eval_['valid_acc'].append(valid_acc)

        return self


In [ ]:
n_epochs = 50


ann = neuralnetwork(n_hidden1=70,n_hidden2=100,
                  l2=0.03, 
                  epochs=n_epochs, 
                  eta=0.0005,
                  minibatch_size=1000, 
                  shuffle=True,
                  seed=1)

In [ ]:
ann.fit(X_train=X_train[:600000], 
       y_train=y_train[:600000],
       X_valid=X_train[600000:],
       y_valid=y_train[600000:])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(range(ann.epochs), ann.eval_['cost'])
plt.ylabel('Cost')
plt.xlabel('Epochs')
plt.legend('Relu-Sigmoid')
#plt.savefig('images/12_07.png', dpi=300)
plt.show()


In [ ]:
plt.plot(range(ann.epochs), ann.eval_['train_acc'], 
         label='training')
plt.plot(range(ann.epochs), ann.eval_['valid_acc'], 
         label='test', linestyle='--')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
#plt.savefig('images/12_08.png', dpi=300)
plt.show()

In [ ]:
acc = (np.sum(y_test == y_test_pred).astype(np.float) / X_test.shape[0])
print("Total accuracy: %.2f%%" % (acc*100))

In [ ]:
miscl_img = X_test[y_test != y_test_pred][:24]
correct_lab = y_test[y_test != y_test_pred][:24]
miscl_lab = y_test_pred[y_test != y_test_pred][:24]

fig, ax = plt.subplots(nrows=4, ncols=6, sharex=True, sharey=True,)
ax = ax.flatten()
for i in range(24):
    img = miscl_img[i].reshape(32, 32)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_title('%d) t: %d p: %d' % (i+1, correct_lab[i], miscl_lab[i]))

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()

In [ ]:
 def confusion_matrix(self, data_array, labels):
        cm = {}
        for i in range(len(data_array)):
            res = self.run(data_array[i])
            res_max = res.argmax()
            target = labels[i][0]
            if (target, res_max) in cm:
                cm[(target, res_max)] += 1
            else:
                cm[(target, res_max)] = 1
        return cm

In [ ]:
from sklearn.metrics import confusion_matrix
# Compute confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
print(cm)

# Show confusion matrix in a separate window
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
class neuralnetwork2(object):
    """ Feedforward neural network / Multi-layer perceptron classifier"""
    
    def __init__(self, n_hidden=30,
                 l2=0., epochs=100, eta=0.001,
                 shuffle=True, minibatch_size=1, seed=None):

        self.random = np.random.RandomState(seed)
        self.n_hidden = n_hidden
        self.l2 = l2
        self.epochs = epochs
        self.eta = eta
        self.shuffle = shuffle
        self.minibatch_size = minibatch_size

    def _onehot(self, y, n_classes):
        """Encode labels into one-hot representation"""
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.
        return onehot.T

    def _sigmoid(self, z):
        """Compute logistic function (sigmoid)"""
        return 1 / (1 + np.exp(-z))

    def _forward(self, X):
        """Compute forward propagation step"""
   
        z_h = np.dot(X, self.w_h) + self.b_h

    
        a_h = self._sigmoid(z_h)

    

        z_out = np.dot(a_h, self.w_out) + self.b_out

    
        a_out = self._sigmoid(z_out)

        return z_h, a_h, z_out, a_out

    def _cost(self, y_enc, output):
        """Compute cost function"""
        L2_term = (self.l2 *
                   (np.sum(self.w_h ** 2.) +
                    np.sum(self.w_out ** 2.)))

        term1 = -y_enc * (np.log(output))
        term2 = (1. - y_enc) * np.log(1. - output)
        cost = np.sum(term1 - term2) + L2_term
        return cost

    def predict(self, X):
        """Predict class labels"""
        z_h, a_h, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)
        return y_pred

    def fit(self, X_train, y_train, X_test, y_test):
        """ Learn weights from training data"""

        n_output = np.unique(y_train).shape[0]  # number of class labels
        n_features = X_train.shape[1]


        
        self.b_h = np.zeros(self.n_hidden)
        self.w_h = self.random.normal(loc=0.0, scale=0.1,
                                      size=(n_features, self.n_hidden))

        self.b_out = np.zeros(n_output)
        self.w_out = self.random.normal(loc=0.0, scale=0.1,
                                        size=(self.n_hidden, n_output))

        epoch_strlen = len(str(self.epochs)) 
        self.eval_ = {'cost': [], 'train_acc': [], 'test_acc': []}

        y_train_enc = self._onehot(y_train, n_output)

      
        for i in range(self.epochs):

            # minibatches
            indices = np.arange(X_train.shape[0])
            print("indices:",indices)

            if self.shuffle:
                self.random.shuffle(indices)

            for start_idx in range(0, indices.shape[0] - self.minibatch_size +
                                   1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]

                #  Calling forward propagation
                z_h, a_h, z_out, a_out = self._forward(X_train[batch_idx])

              
                # Backpropagation
              
                sigma_out = a_out - y_train_enc[batch_idx]

                sigmoid_derivative_h = a_h * (1. - a_h)

                sigma_h = (np.dot(sigma_out, self.w_out.T) *
                           sigmoid_derivative_h)

                grad_w_h = np.dot(X_train[batch_idx].T, sigma_h)
                grad_b_h = np.sum(sigma_h, axis=0)
             
                grad_w_out = np.dot(a_h.T, sigma_out)
                grad_b_out = np.sum(sigma_out, axis=0)


                delta_w_h = (grad_w_h + self.l2*self.w_h)
                delta_b_h = grad_b_h # bias is not regularized
                self.w_h -= self.eta * delta_w_h
                self.b_h -= self.eta * delta_b_h

                delta_w_out = (grad_w_out + self.l2*self.w_out)
                delta_b_out = grad_b_out  # bias is not regularized
                self.w_out -= self.eta * delta_w_out
                self.b_out -= self.eta * delta_b_out

            # Evaluation
        
            z_h, a_h, z_out, a_out = self._forward(X_train)
            
            cost = self._cost(y_enc=y_train_enc,
                                      output=a_out)

            y_train_pred = self.predict(X_train)
            y_test_pred = self.predict(X_test)

            train_acc = ((np.sum(y_train == y_train_pred)).astype(np.float) /
                         X_train.shape[0])
            test_acc = ((np.sum(y_test == y_test_pred)).astype(np.float) /
                         X_test.shape[0])
            print("testac",test_acc)
            sys.stderr.write('\r%0*d/%d | Cost: %.2f '
                             '| Train/Test Acc.: %.2f%%/%.2f%% ' %
                             (epoch_strlen, i+1, self.epochs, cost,
                              train_acc*100, test_acc*100))
            sys.stderr.flush()

            self.eval_['cost'].append(cost)
            self.eval_['train_acc'].append(train_acc)
            self.eval_['test_acc'].append(test_acc)
  
        return self



In [ ]:
n_epochs = 120


ann = neuralnetwork2(n_hidden=150, 
                  l2=0.01, 
                  epochs=n_epochs, 
                  eta=0.005,
                  minibatch_size=100, 
                  shuffle=True,
                  seed=1)

In [ ]:
ann.fit(X_train=X_train[:600000], 
       y_train=y_train[:600000],
       X_test=X_train[600000:],
       y_test=y_train[600000:])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(ann.epochs), ann.eval_['cost'])
plt.ylabel('Cost')
plt.xlabel('Epochs')

plt.show()

plt.plot(range(ann.epochs), ann.eval_['train_acc'], 
         label='training')
plt.plot(range(ann.epochs), ann.eval_['test_acc'], 
         label='test', linestyle='--')
plt.ylabel('Accuracy')

plt.xlabel('Epochs')
plt.legend()
plt.show()


In [ ]:
y_test_pred = ann.predict(X_test)
acc = (np.sum(y_test == y_test_pred).astype(np.float) / X_test.shape[0])
print("Total accuracy: %.2f%%" % (acc*100))

In [ ]:
from sklearn.metrics import confusion_matrix


# Compute confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
print(cm)

# Show confusion matrix in a separate window
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
